In [ ]:
%pip install langchain



In [4]:
!export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."

# import getpass
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
%pip install -qU langchain-openai

In [5]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Carlos")])

AIMessage(content='Hello Carlos! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-03539213-3186-412b-ab60-3bc9c40ad9b2-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [7]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not know your name as I am an AI digital assistant and do not have access to personal information.", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 12, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-94da891a-ad2a-4a43-bf13-ae3f68322041-0', usage_metadata={'input_tokens': 12, 'output_tokens': 26, 'total_tokens': 38})

In [13]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Carlos"),
        AIMessage(content="Hello Carlos! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
        AIMessage(content="Your name is Carlos?"),
        HumanMessage(content="No, my name has changed. Now, my name is Karlutxo?"),
        HumanMessage(content="What's my name and What was my previous name?"),

    ]
)

AIMessage(content='Your current name is Karlutxo and your previous name was Carlos.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 79, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-425845e3-ffb0-4296-973a-c2ab3a018681-0', usage_metadata={'input_tokens': 79, 'output_tokens': 14, 'total_tokens': 93})

Using Message History

In [17]:
%pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 21.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.6
    Uninstalling langchain-core-0.2.6:
      Successfully uninstalled langchain-core-0.2.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.4
    Uninstalling langchain-0.2.4:
      Successfully uninstalled langchain-0.2.4
Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [19]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Charles")],
    config=config,
)

response.content

Parent run c07de343-1a03-428a-93b1-100038c42a1f not found for run fbad06a7-a00f-4a1c-b6a9-c83d53d5f504. Treating as a root run.


'Hello Charles! How can I assist you today?'

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 19c0f2a5-d4f1-4970-b066-cbdceda42510 not found for run 2dca5c62-3fc1-4ada-823d-d9ba1d1f7f58. Treating as a root run.


'Your name is Charles.'

In [21]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 676c8b22-dace-4665-9afe-8761dd87225e not found for run c55a4df6-d4f7-4ced-a29c-5ed4e365355c. Treating as a root run.


"I'm sorry, I do not know your name as I am an AI assistant and do not have personal information about users. How can I assist you today?"

In [22]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 3fbd02d9-0934-47a8-8b53-ca051295ad54 not found for run 77473d93-613f-4aaf-a3c7-c20861c5f841. Treating as a root run.


'Your name is Charles.'

Prompt templates

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello, Bob! How can I assist you today?'

In [25]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [26]:
config = {"configurable": {"session_id": "abc5"}}

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

Parent run 06c8b518-9b46-4948-8b9b-6baf66d9d69b not found for run fad5f6b4-5caf-4c47-8632-f5b46175c80f. Treating as a root run.


'Hello, Jim! How can I assist you today?'

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 9731da7d-de6f-49df-a463-2471331b3e42 not found for run d0f9fe89-9e95-40bd-825d-722a515efc01. Treating as a root run.


'Your name is Jim. How can I assist you further, Jim?'

In [29]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [30]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

Managing Conversation History

In [31]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [32]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [33]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to personal information."

In [34]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'Your favorite ice cream flavor is vanilla.'

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 2943ab05-19a0-480b-bdb1-537af24245c5 not found for run e5a5a856-b92d-46a3-a3ab-6efa530c4a30. Treating as a root run.


"I'm sorry, I do not have access to that information."

In [40]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 52794292-4c5c-43a4-9412-19feb8df5791 not found for run f3f8195c-f3ab-4f30-9ac1-50d1b8220225. Treating as a root run.


"I'm sorry, Todd, but as an AI assistant, I don't have access to personal information about you, such as your favorite ice cream flavor. Why don't you tell me what your favorite ice cream flavor is?"

Streaming

In [41]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Parent run e82dbc34-dac5-40b6-9f16-dc6dcfbfbd39 not found for run 1615b501-8385-4a52-b97c-893b82ce3da8. Treating as a root run.


|Hi| Todd|!| Sure|,| here|'s| another| joke| for| you|:| Why| did| the| math| book| look| sad|?| Because| it| had| too| many| problems|.| 😄||